In [7]:
import asyncio
import aiohttp
import time
from collections import Counter


In [8]:
# Provide 500+ URLs or generate dummy test URLs
URLS = [f"https://httpbin.org/get?i={i}" for i in range(600)]  # 600 test requests


In [9]:
async def fetch(session, url):
    try:
        async with session.get(url) as response:
            status = response.status
            _ = await response.text()
            return status
    except Exception:
        return "EXC"  # Exception


In [10]:


async def stress_test(urls, concurrency=200, burst_per_sec=500):
    sem = asyncio.Semaphore(concurrency)
    results = []

    async with aiohttp.ClientSession() as session:

        async def limited_fetch(url):
            async with sem:
                return await fetch(session, url)

        tasks = []
        start = time.perf_counter()

        for i, url in enumerate(urls):
            tasks.append(asyncio.create_task(limited_fetch(url)))

            # Burst-rate throttling: enforce ~burst_per_sec
            if (i + 1) % burst_per_sec == 0:
                await asyncio.sleep(1)

        for t in tasks:
            results.append(await t)

        duration = time.perf_counter() - start

    return results, duration


In [11]:
from typing import Counter


results, duration = await stress_test(URLS, concurrency=300, burst_per_sec=500)

counts = Counter(results)
total = len(results)
rps = total / duration

print("===== STRESS TEST SUMMARY =====")
print(f"Total Requests: {total}")
print(f"Duration: {duration:.2f}s")
print(f"Throughput: {rps:.2f} req/sec\n")

print("Status Code Breakdown:")
for code, count in counts.items():
    print(f"  {code}: {count}")

if any(code in counts for code in [429, 500, 503, "EXC"]):
    print("\n⚠️ Rate limit or server saturation detected!")
else:
    print("\n✅ No rate-limit detected.")


===== STRESS TEST SUMMARY =====
Total Requests: 600
Duration: 6.48s
Throughput: 92.58 req/sec

Status Code Breakdown:
  503: 600

⚠️ Rate limit or server saturation detected!
